### Maulidio Farhan Rizkullah
### 2141720041
### 19
### 3F

### Kode Pada Praktikum 2

In [3]:
# import library
import tensorflow as tf
import numpy as np
import os
import time

In [4]:
# Mendapatkan path file untuk dataset Shakespeare dari URL
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


1115394/1115394 [==============================] - 0s 0us/step


In [5]:
# Membaca isi file dan mendekodekannya sebagai teks dengan encoding utf-8
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Mencetak panjang teks dalam karakter
print(f'Length of text: {len(text)} characters')


Length of text: 1115394 characters


In [6]:
# Mencetak 250 karakter pertama dari teks
print(text[:250])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
# Membuat kamus dari karakter unik dalam teks dan mengurutkannya
vocab = sorted(set(text))

# Mencetak jumlah karakter unik dalam teks
print(f'{len(vocab)} unique characters')


65 unique characters


In [8]:
# Memecah teks contoh menjadi karakter-karakter menggunakan Unicode split
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

# Menampilkan hasil pemecahan karakter
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
# Membuat layer StringLookup untuk mengonversi karakter menjadi ID
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)


In [10]:
# Menggunakan layer StringLookup untuk mengonversi karakter menjadi ID
ids = ids_from_chars(chars)

# Menampilkan hasil konversi
ids


<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [11]:
# Membuat layer StringLookup untuk mengonversi ID kembali menjadi karakter
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)


In [12]:
# Menggunakan layer StringLookup untuk mengonversi ID kembali menjadi karakter
chars = chars_from_ids(ids)

# Menampilkan hasil konversi
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [13]:
# Menggunakan tf.strings.reduce_join untuk menggabungkan karakter-karakter kembali menjadi teks
tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
# Mendefinisikan fungsi untuk mengonversi ID kembali menjadi teks
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


In [15]:
# Menggunakan layer StringLookup untuk mengonversi semua karakter dalam teks menjadi ID
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

# Menampilkan hasil konversi
all_ids


<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [16]:
# Membuat dataset dari tensor ID
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


In [15]:
# Menampilkan karakter-karakter pertama dalam 10 tensor ID
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [17]:
# Menentukan panjang urutan yang diinginkan
seq_length = 100

In [18]:
# Membuat urutan dari dataset dengan panjang yang ditentukan
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# Menampilkan urutan karakter pertama dalam satu batch
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [19]:
# Menampilkan teks dari lima urutan pertama
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [20]:
# Mendefinisikan fungsi untuk membagi input dan target dari suatu urutan
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text


In [21]:
# Memanggil fungsi pada contoh urutan
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:

# Membuat dataset dari urutan dengan input dan target yang terpisah
dataset = sequences.map(split_input_target)

In [23]:
# Menampilkan contoh input dan target dari dataset
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [24]:
# Ukuran batch
BATCH_SIZE = 64

# Ukuran buffer untuk mengacak dataset
# (TF data dirancang untuk bekerja dengan urutan yang mungkin tak terbatas,
# sehingga tidak mencoba untuk mengacak seluruh urutan dalam memori. Sebaliknya,
# ia menjaga buffer di mana ia mengacak elemen).
BUFFER_SIZE = 10000

# Membuat dataset dengan mengacak, mengelompokkan, dan memuat dengan prefetch
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

# Menampilkan dataset
dataset


<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [25]:
# Panjang kamus dalam layer StringLookup
vocab_size = len(ids_from_chars.get_vocabulary())

# Dimensi embedding
embedding_dim = 256

# Jumlah unit RNN
rnn_units = 1024

In [26]:
# Mendefinisikan kelas model yang merupakan turunan dari tf.keras.Model
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    # Lapisan embedding untuk mengonversi ID menjadi vektor embedding
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # Lapisan GRU (Gated Recurrent Unit) untuk pemrosesan urutan
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    # Lapisan Dense untuk menghasilkan output final
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    # Melakukan embedding pada input
    x = self.embedding(x, training=training)
    # Mendapatkan nilai awal state GRU jika tidak ada
    if states is None:
      states = self.gru.get_initial_state(x)
    # Melakukan proses GRU pada urutan input
    x, states = self.gru(x, initial_state=states, training=training)
    # Melakukan lapisan Dense pada hasil GRU
    x = self.dense(x, training=training)

    # Mengembalikan hasil dan state jika diperlukan
    if return_state:
      return x, states
    else:
      return x

In [27]:
# Membuat objek model menggunakan kelas yang telah didefinisikan
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [28]:
# Melakukan prediksi pada satu batch contoh input dari dataset
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [29]:
# Menampilkan ringkasan arsitektur model
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# Menghasilkan indeks teracak dari distribusi prediksi
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [31]:
sampled_indices

array([41, 63, 50, 26, 38, 40, 29, 11, 31, 34, 65, 46, 14, 65, 21, 39, 57,
       39, 16, 55,  2, 60, 41, 12,  6,  3, 31, 58, 43, 14, 50, 29, 28, 18,
       34, 43, 44, 18, 11, 30, 21,  7, 16, 19, 32, 54,  2, 36, 43, 22, 29,
        8, 38, 24,  8, 32, 63, 14, 36, 15, 52, 55, 21, 16, 38, 21,  0,  9,
       10, 15,  0, 32, 29, 22, 13, 53, 52, 10, 21, 12, 63, 29, 28, 56, 56,
       13, 55, 27, 12,  2, 29, 49, 25, 27, 18, 16,  2,  4, 13,  6])

In [32]:
# Menampilkan contoh input dan prediksi karakter berikutnya
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' what makes robbers bold but too much lenity?\nBootless are plaints, and cureless are my wounds;\nNo w'

Next Char Predictions:
 b"bxkMYaP:RUzgAzHZrZCp ub;'!RsdAkPOEUdeE:QH,CFSo WdIP-YK-SxAWBmpHCYH[UNK].3B[UNK]SPI?nm3H;xPOqq?pN; PjLNEC $?'"


In [33]:
# Menggunakan SparseCategoricalCrossentropy sebagai fungsi loss
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [34]:
# Menghitung rata-rata loss pada contoh batch
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1901917, shape=(), dtype=float32)


In [35]:
# Menghitung eksp dari rata-rata loss
tf.exp(example_batch_mean_loss).numpy()

66.03545

In [36]:
# Mengompilasi model dengan optimizer Adam dan fungsi loss SparseCategoricalCrossentropy
model.compile(optimizer='adam', loss=loss)

In [37]:
# Direktori tempat checkpoint akan disimpan
checkpoint_dir = './training_checkpoints'
# Nama file checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# Callback untuk menyimpan checkpoint
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [38]:
# Jumlah epoch
EPOCHS = 10

In [39]:
# Melatih model dengan dataset dan menyimpan checkpoint
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 13s 53ms/step - loss: 2.7326
Epoch 2/10
172/172 [==============================] - 10s 51ms/step - loss: 1.9956
Epoch 3/10
172/172 [==============================] - 11s 52ms/step - loss: 1.7167
Epoch 4/10
172/172 [==============================] - 12s 53ms/step - loss: 1.5548
Epoch 5/10
172/172 [==============================] - 11s 53ms/step - loss: 1.4549
Epoch 6/10
172/172 [==============================] - 11s 54ms/step - loss: 1.3857
Epoch 7/10
172/172 [==============================] - 11s 55ms/step - loss: 1.3337
Epoch 8/10
172/172 [==============================] - 12s 55ms/step - loss: 1.2889
Epoch 9/10
172/172 [==============================] - 11s 56ms/step - loss: 1.2487
Epoch 10/10
172/172 [==============================] - 12s 57ms/step - loss: 1.2093


### Tugas

In [40]:
# Mendefinisikan kelas CustomTraining yang merupakan turunan dari kelas MyModel
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        # Mendapatkan input dan label dari batch dataset
        inputs, labels = inputs
        # Membuka GradientTape untuk mencatat operasi dalam perhitungan gradien
        with tf.GradientTape() as tape:
            # Mengeksekusi model pada input untuk mendapatkan prediksi
            predictions = self(inputs, training=True)
            # Menghitung nilai kerugian antara prediksi dan label
            loss = self.loss(labels, predictions)
        # Menghitung gradien dari nilai kerugian terhadap parameter model
        grads = tape.gradient(loss, self.trainable_variables)
        # Menggunakan optimizer untuk menerapkan gradien dan memperbarui parameter model
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        # Mengembalikan nilai kerugian sebagai output dari langkah pelatihan
        return {'loss': loss}


In [41]:
# Membuat objek model CustomTraining
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [42]:
# Mengompilasi model dengan optimizer Adam dan fungsi kerugian SparseCategoricalCrossentropy
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [43]:
# Melatih model untuk satu epoch menggunakan dataset
model.fit(dataset, epochs=1)

172/172 [==============================] - 15s 58ms/step - loss: 2.7086


In [44]:
# Jumlah epoch tambahan yang ingin dilatih
EPOCHS = 10

In [45]:
# Menggunakan metrik Mean untuk menghitung rata-rata kerugian
mean = tf.metrics.Mean()

# Melakukan pelatihan selama beberapa epoch
for epoch in range(EPOCHS):
    start = time.time()

    # Mereset nilai metrik Mean untuk setiap epoch
    mean.reset_states()

    # Iterasi melalui setiap batch dalam dataset
    for (batch_n, (inp, target)) in enumerate(dataset):
        # Mengambil log hasil dari satu langkah pelatihan
        logs = model.train_step([inp, target])
        # Memperbarui nilai metrik Mean dengan nilai kerugian
        mean.update_state(logs['loss'])

        # Menampilkan informasi setiap 50 batch
        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # Menyimpan model setiap 5 epoch
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

# Menyimpan model setelah pelatihan selesai
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1927
Epoch 1 Batch 50 Loss 2.0528
Epoch 1 Batch 100 Loss 1.9181
Epoch 1 Batch 150 Loss 1.8359

Epoch 1 Loss: 1.9838
Time taken for 1 epoch 13.14 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8187
Epoch 2 Batch 50 Loss 1.7773
Epoch 2 Batch 100 Loss 1.7005
Epoch 2 Batch 150 Loss 1.6109

Epoch 2 Loss: 1.7087
Time taken for 1 epoch 11.92 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5734
Epoch 3 Batch 50 Loss 1.5550
Epoch 3 Batch 100 Loss 1.5859
Epoch 3 Batch 150 Loss 1.4878

Epoch 3 Loss: 1.5490
Time taken for 1 epoch 12.04 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4409
Epoch 4 Batch 50 Loss 1.4275
Epoch 4 Batch 100 Loss 1.5038
Epoch 4 Batch 150 Loss 1.4089

Epoch 4 Loss: 1.4515
Time taken for 1 epoch 10.92 sec
_____________________________________________________________________

### Perbedaan dengan praktikum 2

Pada praktikum 2 pelatihan menggunakan metode model.fit langsung pada dataset. Fungsi SparseCategoricalCrossentropy digunakan sebagai fungsi loss. Sedangkan pada tugas, model dilatih dengan menggunakan langkah-langkah pelatihan custom yang diimplementasikan dalam class CustomTraining. Proses ini dilakukan dalam loop training dengan menggunakan metrik mean untuk menghitung rata-rata loss. Dalam class CustomTraining juga terdapat tf.GradientTape yang digunakan untuk mencatat operasi dan menghitung gradien. Hal ini memberikan lebih banyak kendali terhadap proses training. Pada praktikum 2 lebih menitikberatkan metode bawaan model.fit, sedangkan tugas memberikan fleksibilitas lebih besar dengan implementasi langkah custom training dalam class CustomTraining.